# Generative AI Optimization Examples
This notebook demonstrates several prompt optimization techniques using the OpenAI API. You'll need an API key to run these examples.

> ⚠️ Make sure you have installed the `openai` Python package and set your `OPENAI_API_KEY` environment variable or configure it manually in the notebook.

In [5]:
# Setup
from openai import OpenAI

client = OpenAI()
import os

# Optionally set your API key here if not using environment variables
# openai.api_key = 'your-api-key-here'

## Example 1: Prompt Compression
Reducing unnecessary instructions from the input prompt.

In [6]:
# Verbose prompt
verbose_prompt = '''
You're an intelligent and helpful assistant. Please help me answer this question in a clear, concise, and professional manner.
The question is: How can I reduce the latency of my GPT-4 application in production?
'''

response = client.chat.completions.create(model="gpt-4",
messages=[{"role": "user", "content": verbose_prompt}],
temperature=0.7)
print(response.choices[0].message.content)

There are several strategies you can use to reduce the latency of your GPT-4 application in production:

1. **Optimization of Model Inference**: Use techniques like quantization, pruning, or distillation to reduce the size of your model without a significant drop in performance. This can help speed up the inference time.

2. **Parallel Processing**: If your application performs multiple inferences at once, you might benefit from parallelizing these tasks. This can be achieved by using multi-threading or distributed computing techniques.

3. **GPU Utilization**: Ensure your application is utilizing GPU for model inference, as this can significantly speed up the process compared to using a CPU.

4. **Using Edge Computing**: If latency is due to network issues, consider deploying your model closer to the source of the data. This can be achieved by using edge computing solutions.

5. **Optimizing your Code**: Look for bottlenecks in your code. This can be done through profiling. Once ident

In [7]:
# Compressed prompt
compressed_prompt = "How can I reduce GPT-4 latency in production?"

response = client.chat.completions.create(model="gpt-4",
messages=[{"role": "user", "content": compressed_prompt}],
temperature=0.7)
print(response.choices[0].message.content)

As of the time this text is written, GPT-4 has not been released, so it's not possible to provide specific advice on how to reduce its latency in production. However, here are some general tips that can be applied to any large-scale AI model (like GPT-3 or future GPT-4):

1. Optimize the Model: If the model is too complex, it would take more time to generate the output. Simplify the model if possible. Some models can be pruned to reduce their size and complexity without significantly affecting their performance.

2. Hardware Acceleration: Use faster and more powerful hardware. GPUs are specifically designed for the kind of parallel processing that AI models require, and they can significantly reduce latency.

3. Use Model Quantization: Model quantization is a technique to reduce the precision of weights in the model. This can reduce the computation and storage requirements and improve inference speed.

4. Batch Predictions: Instead of making predictions one by one, group them into batc

## Example 2: Output Compression
Limit the size and verbosity of the model's response.

In [8]:
# Unconstrained response
prompt = "Tell me about photosynthesis."
response = client.chat.completions.create(model="gpt-3.5-turbo",
messages=[{"role": "user", "content": prompt}])
print(response.choices[0].message.content)

Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, typically from the sun, into chemical energy in the form of glucose. This process is essential for the survival of most living organisms on Earth, as it is the primary way in which energy from the sun is captured and stored in organic compounds that can be used as a source of food and energy.

During photosynthesis, plants absorb sunlight through chlorophyll, a green pigment found in their cells. This light energy is used to convert carbon dioxide from the air and water from the soil into glucose and oxygen. The chemical equation for photosynthesis is:

6CO2 + 6H2O + light energy → C6H12O6 + 6O2

In addition to producing glucose and oxygen, photosynthesis also plays a crucial role in the carbon cycle, as it removes carbon dioxide from the atmosphere and helps regulate the Earth's climate.

Photosynthesis is a complex biochemical process that occurs in two main stages: the light-dependent

In [9]:
# Constrained response
prompt = "Tell me about photosynthesis."
response = client.chat.completions.create(model="gpt-3.5-turbo",
messages=[
    {"role": "system", "content": "Answer in 3 short bullet points."},
    {"role": "user", "content": prompt}
],
max_tokens=100)
print(response.choices[0].message.content)

- Photosynthesis is the process by which plants, algae, and some bacteria convert light energy into chemical energy to produce glucose.
- It occurs in chloroplasts, the organelles within plant cells that contain the pigment chlorophyll.
- During photosynthesis, carbon dioxide and water are converted into glucose and oxygen through a series of complex chemical reactions.
